In [14]:
from matplotlib import pyplot as plt
plt.style.use('bmh')
%matplotlib inline

import numpy as np
from astropy import units as u
from astropy.utils import console

from piaa import observation
from astropy.io import fits

In [24]:
dirs = {
    'New Moon': '/var/panoptes/images/fields/HatP20/ee04d1/20170129T060339/',
    'Full Moon': '/var/panoptes/images/fields/HatP20/ee04d1/20170131T053503/',
}

In [3]:
plt.figure(figsize=(15,12))

for moon, directory in dirs.items():

    obs = observation.Observation(directory)

    frame_index = 0

    exp_time = obs.get_header_value(frame_index, 'EXPTIME') * u.second
    print("Exposure time: {}".format(exp_time))

    iso = obs.get_header_value(frame_index, 'ISO')
    print("ISO: {}".format(iso))

    gain = 2.0 * (u.electron / u.adu)
    print("Gain: {}".format(gain))

    readout_noise_e = 11.5 * (u.electron)
    readout_noise_adu = readout_noise_e / gain
    print("Readout noise [e⁻]: {}".format(readout_noise_e))
    print("Readout noise [ADU]: {}".format(readout_noise_adu))

    # We want photon noise >= readout noise (in electrons). That minimum is:
    photon_noise_equal_read_noise = readout_noise_adu * readout_noise_e.value
    print("RN = PN: {}".format(photon_noise_equal_read_noise))

    moon_fraction = obs.get_header_value(frame_index, 'MOONFRAC')
    print("Moon: {:.02f}".format(moon_fraction))

    for frame_index in console.ProgressBar(range(len(obs.files)), ipython_widget=True):
        obs.get_frame_background(frame_index, summary=True)

    background_data =np.array(obs.hdf5['background'])

    plt.plot(background_data[:, 0, 0], 'r', label='R Channel')
    plt.plot(background_data[:, 1, 0], 'g', label='G Channel')
    plt.plot(background_data[:, 2, 0], 'b', label='B Channel')

plt.title("Background Levels")
plt.legend()
plt.xlabel('Frame')
plt.ylabel('Count [ADU]')

In [ ]:
minimum_counts_per_pixsec = None
for channel, data in zip(['R', 'G', 'B'], frame_back):
    print("{} pixels:".format(channel))
    print("\tBackground Median: {:.02f}".format(data[0]))
    print("\tBackground RMS Median: {:.02f}".format(data[1]))
    counts_per_pixsec = (np.ma.median(data) / exp_time) * u.adu
    if minimum_counts_per_pixsec is None or counts_per_pixsec < minimum_counts_per_pixsec:
        minimum_counts_per_pixsec = counts_per_pixsec
    print("\tMedian Counts per Second per Pixel: {:.02f}".format(counts_per_pixsec))
    print("\tMedian Electrons per Second per Pixel: {:.02f}".format(counts_per_pixsec * gain))
    print("")

In [ ]:
# We want the minimum counts (given by the blue) to be greater than min_electrons. To do this, we need to expose for
min_exp_time = photon_noise_equal_read_noise / (minimum_counts_per_pixsec)
min_exp_time